In [1]:
#connect to google drive
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [2]:
#load corr matrix from file
import pickle
import numpy as np
all_corr = pickle.load(open('/content/drive/My Drive/correlation_matrix/correlations_filecc400.pkl', 'rb'))

In [3]:
#divide whole collection to normal and autistic
normals = {}
autistic = {}
for item in all_corr:
  if(all_corr[item][1] == 0):
    normals[item] = all_corr[item]
  else:
    autistic[item] = all_corr[item]

In [4]:
#sum over nomal
sum_n = np.zeros(76636)
for item_n in normals:
  sum_n = sum_n + normals[item_n][0]

#sum over autistic
sum_a = np.zeros(76636)
for item_a in autistic:
  sum_a = sum_a + autistic[item_a][0]

print(np.shape(sum_n))

(76636,)


In [5]:
#make diff matrix
diff = sum_n - sum_a
print(np.shape(diff))
print(sum_n)

(76636,)
[-63.62773595  -5.08491527 -38.35125636 ... 146.46137406  21.59880354
 -19.2785524 ]


In [6]:
#add index to diff matrix
num_diff = []
for counter in range(len(diff)):
  num_diff = num_diff + [[counter ,diff[counter]]]

print(num_diff)

Output hidden; open in https://colab.research.google.com to view.

In [7]:
#sort diff matrix
arr = np.array(num_diff)
sorted_diff = arr[arr[:,1].argsort()]

In [8]:
#make a mask from first and last 1000 elements
mask1 = sorted_diff[0:1000,0]
mask2 = sorted_diff[-1000:,0]
mask = np.concatenate((mask1, mask2))
mask = mask.astype('int32')
print(mask)

[75371 42870 42922 ...  8873 25959 65585]


In [9]:
# apply mask to data
masked_diff = np.array(num_diff)[mask.astype(int)]
print(masked_diff)

[[ 7.53710000e+04 -4.71657790e+01]
 [ 4.28700000e+04 -4.58432467e+01]
 [ 4.29220000e+04 -4.53570299e+01]
 ...
 [ 8.87300000e+03  6.47094740e+01]
 [ 2.59590000e+04  6.56654748e+01]
 [ 6.55850000e+04  7.33437897e+01]]


In [10]:
import pandas as pd
from sklearn.model_selection import train_test_split

import tensorflow as tf
from keras import Sequential
from keras.layers import Dense
from sklearn.metrics import confusion_matrix
from sklearn.preprocessing import StandardScaler

X = []
for ff in all_corr:
  tempx = all_corr[ff][0]
  masked_tempx = np.array(tempx)[mask.astype(int)]
  X.append(list(masked_tempx))
X = pd.DataFrame(X)
sc = StandardScaler()
X = sc.fit_transform(X)
Y = []
for jj in all_corr:
  Y.append(all_corr[jj][1])
Y = np.asarray(Y)

print(np.shape(X))
print(np.shape(Y))

(884, 2000)
(884,)


In [34]:
import keras
classifier = Sequential()
classifier.add(Dense(2, activation='relu', kernel_initializer='random_normal', input_dim=2000))
# classifier.add(Dense(4, activation='relu', kernel_initializer='random_normal'))
# classifier.add(Dense(2, activation='relu', kernel_initializer='random_normal'))
classifier.add(Dense(1, activation='sigmoid', kernel_initializer='random_normal'))
loss_fn = tf.keras.losses.MeanSquaredError(reduction='sum_over_batch_size')

classifier.compile(optimizer ='adam',loss=loss_fn, metrics =['accuracy'])

accuracy = []
F1 = []
x_train, x_test, y_train, y_test = train_test_split(X, Y, test_size=0.10)

In [38]:
classifier.fit(x_train,y_train, batch_size = 40, epochs = 4)
y_pred=classifier.predict(x_test)

y_pred = np.where(y_pred > 0.5, 1, 0)

cm = confusion_matrix(y_test, y_pred)
accuracy.append((cm[0][0] + cm[1][1])/sum(sum(cm)))
precision = cm[0][0]/(cm[0][0] + cm[1][1])
recall =  cm[0][0]/(cm[0][0] + cm[1][0])
F1.append(2 * precision * recall / (precision + recall))

print('accuracy =', np.mean(accuracy))
print('F1 =', np.mean(F1))

Epoch 1/4
20/20 [==============================] - 0s 2ms/step - loss: 0.1031 - accuracy: 0.8704
Epoch 2/4
20/20 [==============================] - 0s 2ms/step - loss: 0.0961 - accuracy: 0.8742
Epoch 3/4
20/20 [==============================] - 0s 2ms/step - loss: 0.0910 - accuracy: 0.8830
Epoch 4/4
20/20 [==============================] - 0s 2ms/step - loss: 0.0868 - accuracy: 0.8893
accuracy = 0.6966292134831461
F1 = 0.5585792269407119


In [18]:
y_pred = np.where(y_pred > 0.5, 1, 0)

cm = confusion_matrix(y_test, y_pred)
accuracy.append((cm[0][0] + cm[1][1])/sum(sum(cm)))
precision = cm[0][0]/(cm[0][0] + cm[1][1])
recall =  cm[0][0]/(cm[0][0] + cm[1][0])
F1.append(2 * precision * recall / (precision + recall))

print('accuracy =', np.mean(accuracy))
print('F1 =', np.mean(F1))


accuracy = 0.745318352059925
F1 = 0.6464997107198943


In [ ]:
print(cm)

[[28 20]
 [12 29]]


In [ ]:
#get weights from model
for layer in classifier.layers:
    weights += (layer.get_weights()) # list of numpy arrays


In [ ]:
w = pd.DataFrame(list(map(np.ravel, weights)))
w = w.transpose()

w.to_excel("/content/drive/My Drive/parisa's 1 sigmoid.xlsx")